In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import sys
from utils import *

In [2]:
################################# Hypyer parameter #################################
alpha = 0.001
epoch = 2
input_dim = 95    # length of one hot encoded character --> 95
hidden_dim = 95  # number of hidden neuron
n_layers = 1      # number of hidden layers
batch_size = 1
seq_len = 100
np.random.seed(42)

In [3]:
# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

CUDA NOT supported


In [4]:
############# locad data ###############
character_dic = generate_dictionary(['train.txt','val.txt','test.txt'])
train_song_dic = read_song('train.txt')
valid_song_dic = read_song('val.txt')
test_song_dic = read_song('test.txt')

In [10]:
train_song_dic.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [18]:
train_rand_index = np.arange(len(train_song_dic))
np.random.shuffle(train_rand_index)

In [35]:
model = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True).to(computing_device) # input models
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = alpha)

train_loss = []
valid_loss = []

for i in range(epoch):
    train_loss_epoch = []
    valid_loss_epoch = []
    
    # randomize train song index
    train_rand_index = np.arange(len(train_song_dic))
    np.random.shuffle(train_rand_index)
    
    for song_index, song in enumerate(train_rand_index):
#         hidden_state = torch.randn(n_layers, batch_size, hidden_dim)
#         cell_state = torch.randn(n_layers, batch_size, hidden_dim)
        hidden_state = torch.zeros(n_layers, batch_size, hidden_dim)
        cell_state = torch.zeros(n_layers, batch_size, hidden_dim)
        hidden = (hidden_state, cell_state)
        
        # each 1 song at a time
        song_raw = train_song_dic[song]
        
        # convert song to one-hot
        input_lists, target_lists = encode_song(song_raw, character_dic)
        
        for chunk_index, (input_list, target_list) in enumerate(zip(input_lists,target_lists)):
            # We need to clear them out before each instance
            #optimizer.zero_grad()
            
            # convert input, and target to tensor
            input_tensor = torch.FloatTensor(input_list).unsqueeze_(0)
            target_tensor = torch.from_numpy(target_list.argmax(1))
            
            # send all data to GUP
            input_tensor, target_tensor= input_tensor.to(computing_device), target_tensor.to(computing_device)

            # Run our forward pass.
            output, hidden = model(input_tensor,hidden)

            # compute loss, run optimizer step
            loss = loss_function(torch.squeeze(output), target_tensor)
            loss.backward()
            optimizer.step()
            
            # detech hidden state
            hidden = (hidden[0].detach(), hidden[1].detach())
            
            # print status of training
            print('Train song #{}, #chunk #{}, loss={}'.format(song_index,chunk_index,loss.item()))

            # save training loss
            train_loss_epoch.append(loss.item())
  
    train_loss.append(np.array(train_loss_epoch).mean())

Train song #0, #chunk #0, loss=4.557586669921875
Train song #0, #chunk #1, loss=4.552938938140869
Train song #0, #chunk #2, loss=4.531892776489258
Train song #0, #chunk #3, loss=4.530763149261475
Train song #0, #chunk #4, loss=4.516415119171143
Train song #0, #chunk #5, loss=4.5184006690979
Train song #0, #chunk #6, loss=4.5126423835754395
Train song #0, #chunk #7, loss=4.528110980987549
Train song #0, #chunk #8, loss=4.541273593902588
Train song #1, #chunk #0, loss=4.536314487457275
Train song #1, #chunk #1, loss=4.5385050773620605
Train song #1, #chunk #2, loss=4.533474922180176
Train song #2, #chunk #0, loss=4.539168834686279
Train song #2, #chunk #1, loss=4.520561695098877
Train song #2, #chunk #2, loss=4.527151107788086
Train song #2, #chunk #3, loss=4.53338623046875
Train song #2, #chunk #4, loss=4.527917385101318
Train song #2, #chunk #5, loss=4.523868560791016
Train song #2, #chunk #6, loss=4.507065773010254
Train song #2, #chunk #7, loss=4.487095355987549
Train song #2, #chunk

Train song #29, #chunk #0, loss=4.0650835037231445
Train song #29, #chunk #1, loss=4.087253093719482
Train song #29, #chunk #2, loss=4.2078537940979
Train song #29, #chunk #3, loss=3.6955251693725586
Train song #29, #chunk #4, loss=3.534785270690918
Train song #29, #chunk #5, loss=3.6319143772125244
Train song #29, #chunk #6, loss=3.546072006225586
Train song #30, #chunk #0, loss=4.26539421081543
Train song #30, #chunk #1, loss=3.6679847240448
Train song #30, #chunk #2, loss=3.546825647354126
Train song #31, #chunk #0, loss=4.246091842651367
Train song #31, #chunk #1, loss=3.8767688274383545
Train song #31, #chunk #2, loss=3.718038320541382
Train song #31, #chunk #3, loss=3.7639551162719727
Train song #31, #chunk #4, loss=3.6746714115142822
Train song #31, #chunk #5, loss=3.4722559452056885
Train song #32, #chunk #0, loss=4.195574760437012
Train song #32, #chunk #1, loss=3.915135383605957
Train song #32, #chunk #2, loss=3.740466594696045
Train song #32, #chunk #3, loss=3.62170743942260

Train song #63, #chunk #3, loss=3.4708776473999023
Train song #64, #chunk #0, loss=4.194777488708496
Train song #64, #chunk #1, loss=3.989834785461426
Train song #64, #chunk #2, loss=3.5152370929718018
Train song #64, #chunk #3, loss=3.6680185794830322
Train song #64, #chunk #4, loss=3.5150516033172607
Train song #64, #chunk #5, loss=3.334921360015869
Train song #65, #chunk #0, loss=4.171176910400391
Train song #65, #chunk #1, loss=4.14622688293457
Train song #65, #chunk #2, loss=3.4970765113830566
Train song #65, #chunk #3, loss=4.05723762512207
Train song #65, #chunk #4, loss=3.8987374305725098
Train song #65, #chunk #5, loss=3.3865346908569336
Train song #65, #chunk #6, loss=3.746121883392334
Train song #65, #chunk #7, loss=4.006657123565674
Train song #65, #chunk #8, loss=3.3274941444396973
Train song #66, #chunk #0, loss=4.292086124420166
Train song #66, #chunk #1, loss=4.054067611694336
Train song #66, #chunk #2, loss=3.679140090942383
Train song #66, #chunk #3, loss=3.3956339359

Train song #96, #chunk #0, loss=4.204659938812256
Train song #96, #chunk #1, loss=3.7929983139038086
Train song #96, #chunk #2, loss=3.571619987487793
Train song #97, #chunk #0, loss=4.097883224487305
Train song #97, #chunk #1, loss=3.7645068168640137
Train song #97, #chunk #2, loss=3.5908892154693604
Train song #98, #chunk #0, loss=4.067173004150391
Train song #98, #chunk #1, loss=3.5520644187927246
Train song #98, #chunk #2, loss=3.7742362022399902
Train song #98, #chunk #3, loss=3.5503852367401123
Train song #98, #chunk #4, loss=3.435490131378174
Train song #99, #chunk #0, loss=4.170731067657471
Train song #99, #chunk #1, loss=4.14472770690918
Train song #99, #chunk #2, loss=3.5706937313079834
Train song #99, #chunk #3, loss=3.555000066757202
Train song #100, #chunk #0, loss=4.280282497406006
Train song #100, #chunk #1, loss=4.045160293579102
Train song #100, #chunk #2, loss=3.9482507705688477
Train song #100, #chunk #3, loss=3.7131731510162354
Train song #100, #chunk #4, loss=3.414

Train song #129, #chunk #1, loss=4.240320682525635
Train song #129, #chunk #2, loss=3.596151828765869
Train song #129, #chunk #3, loss=4.104828834533691
Train song #129, #chunk #4, loss=3.926466464996338
Train song #130, #chunk #0, loss=4.023027420043945
Train song #130, #chunk #1, loss=4.009822368621826
Train song #130, #chunk #2, loss=4.1402058601379395
Train song #130, #chunk #3, loss=3.5881600379943848
Train song #131, #chunk #0, loss=4.061822414398193
Train song #131, #chunk #1, loss=3.594193696975708
Train song #131, #chunk #2, loss=3.446850061416626
Train song #132, #chunk #0, loss=3.921010971069336
Train song #132, #chunk #1, loss=3.8535661697387695
Train song #132, #chunk #2, loss=3.882411479949951
Train song #132, #chunk #3, loss=4.028425693511963
Train song #132, #chunk #4, loss=3.6460134983062744
Train song #132, #chunk #5, loss=3.727966547012329
Train song #132, #chunk #6, loss=3.6385984420776367
Train song #132, #chunk #7, loss=3.7149829864501953
Train song #132, #chunk #

...

Train song #663, #chunk #1, loss=3.662496328353882
Train song #663, #chunk #2, loss=3.4792709350585938
Train song #664, #chunk #0, loss=3.690880537033081
Train song #664, #chunk #1, loss=3.6609225273132324
Train song #664, #chunk #2, loss=3.4572954177856445
Train song #664, #chunk #3, loss=3.4249179363250732
Train song #664, #chunk #4, loss=3.4941821098327637
Train song #664, #chunk #5, loss=3.464578628540039
Train song #664, #chunk #6, loss=3.4852051734924316
Train song #665, #chunk #0, loss=3.710446834564209
Train song #665, #chunk #1, loss=3.3883109092712402
Train song #665, #chunk #2, loss=3.3488965034484863
Train song #665, #chunk #3, loss=3.456984043121338
Train song #665, #chunk #4, loss=3.461846113204956
Train song #665, #chunk #5, loss=3.247570276260376
Train song #666, #chunk #0, loss=3.7474889755249023
Train song #666, #chunk #1, loss=3.6818113327026367
Train song #666, #chunk #2, loss=3.571587324142456
Train song #666, #chunk #3, loss=3.662418842315674
Train song #666, #chu

Train song #682, #chunk #3, loss=3.0526719093322754
Train song #683, #chunk #0, loss=3.691218852996826
Train song #683, #chunk #1, loss=3.797560691833496
Train song #683, #chunk #2, loss=3.4703855514526367
Train song #683, #chunk #3, loss=3.5139734745025635
Train song #683, #chunk #4, loss=3.5906589031219482
Train song #683, #chunk #5, loss=3.6639139652252197
Train song #683, #chunk #6, loss=3.40438175201416
Train song #683, #chunk #7, loss=3.430920362472534
Train song #683, #chunk #8, loss=3.389502763748169
Train song #683, #chunk #9, loss=3.584960699081421
Train song #683, #chunk #10, loss=3.3757877349853516
Train song #683, #chunk #11, loss=3.502387285232544
Train song #683, #chunk #12, loss=3.563295364379883
Train song #683, #chunk #13, loss=3.4883594512939453
Train song #683, #chunk #14, loss=3.35870361328125
Train song #684, #chunk #0, loss=3.6272690296173096
Train song #684, #chunk #1, loss=3.5422589778900146
Train song #684, #chunk #2, loss=3.222882032394409
Train song #685, #c

Train song #713, #chunk #4, loss=3.8243179321289062
Train song #713, #chunk #5, loss=3.5750532150268555
Train song #713, #chunk #6, loss=3.774531602859497
Train song #713, #chunk #7, loss=3.3068182468414307
Train song #714, #chunk #0, loss=3.751765489578247
Train song #714, #chunk #1, loss=3.53664493560791
Train song #714, #chunk #2, loss=3.2307629585266113
Train song #715, #chunk #0, loss=3.7681586742401123
Train song #715, #chunk #1, loss=3.478921890258789
Train song #715, #chunk #2, loss=3.4176483154296875
Train song #715, #chunk #3, loss=2.8793249130249023
Train song #716, #chunk #0, loss=3.7104263305664062
Train song #716, #chunk #1, loss=3.536728620529175
Train song #716, #chunk #2, loss=3.4386792182922363
Train song #716, #chunk #3, loss=3.9077389240264893
Train song #716, #chunk #4, loss=3.6780381202697754
Train song #716, #chunk #5, loss=3.2622203826904297
Train song #716, #chunk #6, loss=3.6137590408325195
Train song #716, #chunk #7, loss=3.8076517581939697
Train song #716, #

Train song #740, #chunk #2, loss=3.320178747177124
Train song #740, #chunk #3, loss=3.649698495864868
Train song #740, #chunk #4, loss=3.4689269065856934
Train song #740, #chunk #5, loss=3.2666115760803223
Train song #741, #chunk #0, loss=3.5421226024627686
Train song #741, #chunk #1, loss=3.268620491027832
Train song #742, #chunk #0, loss=3.770669937133789
Train song #742, #chunk #1, loss=3.485441207885742
Train song #742, #chunk #2, loss=3.288376569747925
Train song #743, #chunk #0, loss=3.661747932434082
Train song #743, #chunk #1, loss=3.2618486881256104
Train song #744, #chunk #0, loss=3.837507963180542
Train song #744, #chunk #1, loss=3.635422706604004
Train song #744, #chunk #2, loss=3.5098743438720703
Train song #744, #chunk #3, loss=3.448925733566284
Train song #744, #chunk #4, loss=2.9711408615112305
Train song #745, #chunk #0, loss=3.3478057384490967
Train song #746, #chunk #0, loss=3.813643455505371
Train song #746, #chunk #1, loss=3.8892550468444824
Train song #746, #chunk

Train song #773, #chunk #0, loss=3.649216890335083
Train song #773, #chunk #1, loss=3.6743686199188232
Train song #773, #chunk #2, loss=3.4869883060455322
Train song #773, #chunk #3, loss=3.4048454761505127
Train song #773, #chunk #4, loss=3.033015251159668
Train song #774, #chunk #0, loss=3.6494977474212646
Train song #774, #chunk #1, loss=3.637443780899048
Train song #774, #chunk #2, loss=3.519918203353882
Train song #774, #chunk #3, loss=3.415173053741455
Train song #774, #chunk #4, loss=3.2460014820098877
Train song #775, #chunk #0, loss=3.7075095176696777
Train song #775, #chunk #1, loss=3.8153815269470215
Train song #775, #chunk #2, loss=3.60408353805542
Train song #775, #chunk #3, loss=3.3883166313171387
Train song #775, #chunk #4, loss=3.318619728088379
Train song #776, #chunk #0, loss=3.702364444732666
Train song #776, #chunk #1, loss=3.622715950012207
Train song #776, #chunk #2, loss=3.486703872680664
Train song #776, #chunk #3, loss=3.373950242996216
Train song #776, #chunk 